In [30]:
from datetime import datetime, timezone
import orjson
import aiohttp
import asyncio
import pandas as pd
import time

START_TIME = int(datetime(year=2021, month=11, day=1, hour=0, minute=0).timestamp())
END_TIME = int(datetime(year=2021, month=11, day=18, hour=0, minute=0).timestamp())

async def get(url):
	async with aiohttp.ClientSession(json_serialize=orjson.dumps) as session:
		async with session.get(url) as resp:
			data = await resp.json(loads=orjson.loads)
			return data
   
async def determine_tuples(start_time, end_time, granularity=15):
    # return array of times to multiprocess / async.gather
    tups = []
    # max return len is 1500 samples
    while end_time - start_time > 0:
        t = (end_time-(1499*15),end_time)
        
        tups.append(t)
        end_time -= 1500

    return tups

async def aggregate_klines(underlying, granularity, start_time, end_time,  count):

    aggregate = pd.DataFrame(columns=['startTime', 'time','open', 'high', 'low', 'close', 'volume'])
    step = 0
    
    # get current timestep to the nearest round minute to align with ftx candle data aggregation (i think)
    # to give us a starting timestep 

    current_utc_timestamp = time.time()

    most_recent_candle = current_utc_timestamp - (current_utc_timestamp % granularity)

    # instead of calculating the remaining timesteps pre-1st request, just send the request, count how many 
    # instances of OHLC data we get back, deduce that from count (its usually 1500), and run it back turbo.

    while aggregate.shape[0] < count:
        if step == 0:            
            data = await get(url = f"https://ftx.com/api/markets/{underlying}/candles?resolution={granularity}&end_time={most_recent_candle}")
            print(data['result'])
            break
    
        else:
            end = int(most_recent_candle - (granularity * step))
            data = await get(url=f"https://ftx.com/api/markets/{underlying}/candles?resolution={granularity}&end_time={end}") 

        data = data['result'][::-1]
        
        current = pd.DataFrame(data=data)
        aggregate = aggregate.append(current, ignore_index=True)
        step += len(data)
              
    # check if current step is greater than count, and trim off excess data.
        if step > count:
            aggregate = aggregate.iloc[:count+1]
                  
    return aggregate

tups = await determine_tuples(START_TIME, END_TIME,granularity=15)
# await aggregate_klines(underlying='BTC-PERP', granularity=60, count=count)

In [35]:
import pytz

# check to ensure data received is expected timeframe
print(datetime.fromtimestamp(tups[0][0], tz=pytz.utc))
print(datetime.fromtimestamp(tups[0][1], tz=pytz.utc))

data = await get(url = f"https://ftx.com/api/markets/BTC-PERP/candles?resolution=15&start_time={tups[0][0]}&end_time={tups[0][1]}")

print(len(data['result']))
print(data['result'][0]['startTime'])
print(data['result'][-1]['startTime'])


2021-11-17 22:45:15+00:00
2021-11-18 05:00:00+00:00
1500
2021-11-17T22:45:15+00:00
2021-11-18T05:00:00+00:00


In [17]:
a = 1637189100
b = 1637211600

assert b - a + (1500 * 15) == 0 

# print(datetime.fromtimestamp(a))
# print(datetime.fromtimestamp(b))


AssertionError: 

In [13]:
25*4

# print(tups)

100